In [13]:
from urllib.request import urlopen as uReq #urllib will get the page itself
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import nltk
import heapq
import requests
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from selenium import webdriver
import os
import urllib.request
import urllib.parse
from tqdm import tqdm

In [14]:
therapeutic_justice_type = []

In [15]:
website = 'https://www.judiciary.gov.sg/family'
user_agent = {'User-agent' : 'Mozilla/5.0'}
page = requests.get(website, headers=user_agent)
soup = BeautifulSoup(page.text, 'lxml')
fact_cards = soup.find_all('a', {'class': 'nav-link'})
tj_dict = {}
all_types = []

for fact_card in fact_cards[8:]:
    tj_type = fact_card.get_text().split('\n')[1]
    tj_desc = fact_card.get_text().split('\n')[2]
    tj_link = fact_card['href']
    tj_dict[tj_type] = [tj_desc, tj_link]
    all_types.append(tj_type)

tj_dict

{'Adoption': ['Adoption is the process where parties apply to become the adoptive parents of a child. The Family Courts hear applications for adoption orders.',
  '/family/adoption'],
 'Care and protection for children and young persons': ['The Youth Courts may make care and protection orders to ensure that children in need of care and protection are safe.',
  '/family/care-protection-children-young-persons'],
 'Divorce': ['An application for divorce is a legal procedure to end a marriage. The Family Courts hear divorce applications.',
  '/family/divorce'],
 'Deputyship': ['The Family Courts can appoint a deputy to make decisions for a person who lacks mental capacity. Find out what it means and the ways to apply.',
  '/family/deputyship'],
 'Direct judicial communication for international family proceedings affecting children': ['Information for judges in other jurisdictions on whom to contact regarding direct judicial communication with the Family Justice Courts for international fam

In [16]:
# link_dict = {}
# for type in all_types:
#     next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
#     page = requests.get(next_page, headers=user_agent)
#     soup = BeautifulSoup(page.text, 'lxml')
#     tr_detailed_desc = soup.find_all('div', {'class': 'acc-panel'})
#     for desc in tr_detailed_desc:
#         for link in desc.find_all('a'):
#             link_dict[link.string] = link['href']   
# print(link_dict)



In [17]:

# for type in all_types:
#     next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
#     page = requests.get(next_page, headers=user_agent)
#     soup = BeautifulSoup(page.text, 'lxml')
#     tr_detailed_desc = soup.find_all('div', {'class': 'acc-panel'})
#     for desc in tr_detailed_desc:
#         desc_text = desc.get_text()
#         for linktext, link in link_dict.items():
#             if linktext in desc_text:
#                 desc_text = desc_text.replace(linktext, link)
#         print(desc_text)
#         print("---------------------------------------------------------")
        

In [23]:
desc_dict = {}
tdesc_dict = {}
ndesc_dict = {}
ddesc_dict = {}
for type in all_types:
    print(type)
    next_page = 'https://www.judiciary.gov.sg' + tj_dict[type][1]
    page = requests.get(next_page, headers=user_agent)
    soup = BeautifulSoup(page.text, 'lxml')
    accr_desc = soup.find_all('div', {'class': 'accordion-wrapper'})
    for desc in accr_desc:
        desc_text = desc.get_text()
        for link in desc.find_all('a'):
            desc_dict[link.string] = link['href']
        print(desc_text)
    if type =="Adoption" or type == "Protection for vulnerable adults" :
        timeline_desc = soup.find_all('div', {'class': 'timeline'})
        for t_desc in timeline_desc:
            timeline_text = t_desc.get_text()
            for link in t_desc.find_all('a'):
                tdesc_dict[link.string] = link['href']
            print(timeline_text)
    elif type == "Care and protection for children and young persons" or type == "Guardianship":
        nav_desc = soup.find_all('li', {'class': 'nav-item w-100'})
        for n_desc in nav_desc:
            nav_text = n_desc.get_text()
            for link in n_desc.find_all('a'):
                ndesc_dict[nav_text.replace("\n", "")] = link['href']
            print(nav_text)
    else:
        div_desc = soup.find_all('div', {'class': 'row'})
    
        if type != "Direct judicial communication for international family proceedings affecting children" and type != "Syariah Court orders":
            type_list = []
            for d_desc in div_desc[:-2]:
                div_text = d_desc.get_text()
                type_list.append(list(filter(None,div_text.split("\n"))))
                for link in d_desc.find_all('a'):
                    ddesc_dict[link.string] = link['href']
                    print(link['href'])
                print(div_text)
    print("---------------------------------------------------------")

# for desc in tr_detailed_desc:
#     for link in desc.find_all('a'):
#         link_dict[link.string] = link['href']   

Adoption

Before applying to the courts: pre-adoption stepsThe adoption process starts at the MSF. Before you file an application for an adoption order with the courts, you must do all of the following: Check that you meet the requirements to adopt.Complete the pre-adoption steps. There are different steps depending on the nationality of the child you wish to adopt. Refer to the respective MSF guides:        How to adopt a Singaporean or permanent resident child.How to adopt a child from the People's Republic of China. How to adopt a foreign child (excluding children from the People's Republic of China). Refer to the MSF's website for more information on adoption in Singapore.



Relevant legislationLegislation associated with this topic includes:The Adoption of Children Act.The Family Justice Act.Part 6 of the Family Justice Rules.Part 2 of the Fifth Schedule of the Family Justice Rules for the court fees. 

Practice directionsRefer to Part VIII of the Family Justice Courts Practice D

In [19]:
desc_dict

{'meet the requirements to adopt': 'https://www.msf.gov.sg/Adoption/Pages/Who-can-Adopt.aspx',
 'How to adopt a Singaporean or permanent resident child': 'https://www.msf.gov.sg/Adoption/Pages/How-to-adopt-a-citizen-or-PR.aspx',
 "How to adopt a child from the People's Republic of China": 'https://www.msf.gov.sg/Adoption/Pages/How-to-adopt-a-child-from-the-Peoples-Republic-of-China.aspx',
 "How to adopt a foreign child (excluding children from the People's Republic of China)": 'https://www.msf.gov.sg/Adoption/Pages/How-to-adopt-a-foreign-child-excluding-children-from-the-Peoples-Republic-of-China.aspx',
 "MSF's website": 'https://www.msf.gov.sg/Adoption',
 'Adoption of Children Act': 'https://sso.agc.gov.sg/Act/ACA1939',
 'Family Justice Act': 'https://sso.agc.gov.sg/Act/FJA2014',
 'Part 6 of the Family Justice Rules': 'https://sso.agc.gov.sg/SL/FJA2014-S813-2014#P16-',
 'Part 2 of the Fifth Schedule of the Family Justice Rules': 'https://sso.agc.gov.sg/SL/FJA2014-S813-2014#Sc5-',
 'Pa

In [20]:
tdesc_dict

{'File an application for an adoption order': '/family/file-adoption-order',
 'After filing: submit additional documents for adoption ': '/family/submit-additional-documents-adoption-order',
 'Going to court for adoption': '/family/going-to-court-adoption-order',
 'Appeal against a decision by a Family Courts district judge or magistrate': '/family/appeal-against-decision-by-family-courts-judge',
 'Apply to protect a vulnerable adult ': '/family/apply-to-protect-vulnerable-adult',
 'Going to court for a vulnerable adult application': '/family/going-to-court-vulnerable-adult-application'}

In [21]:
ndesc_dict

{'Care and protection orders: overviewFind out when are children or young persons in need of care and protection, and the care and protection orders that the Youth Courts may make. ': '/family/care-protection-order-overview',
 'Understand the care and protection order application processFind out what to expect if an application for a care and protection order is filed for your child.': '/family/understand-care-protection-order-application-process',
 'File a guardianship applicationFind out the steps to file a guardianship application, the estimated fees and the documents you have to prepare.': '/family/guardianship/file-a-guardianship-application',
 'Going to court for a guardianship applicationFind out what to expect if you have to attend court for a guardianship application.': '/family/going-to-court-guardianship-application'}

In [22]:
ddesc_dict

{None: '/family/counselling-psychological-services'}